In [1]:
import sys
from common import commons
home = commons.home
sys.path.append('/home/ec2-user/anaconda3/lib/python3.6/site-packages')
import pandas as pd
import numpy as np
from models import ModelSelection as MS
from models import Ensemble as es
from models import xgbooster
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC,LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_predict,cross_val_score
from sklearn.metrics import confusion_matrix,recall_score,precision_score,accuracy_score,f1_score,roc_curve,roc_auc_score,precision_recall_curve
import matplotlib.pyplot as plt
from sklearn import clone
from sklearn.externals import joblib
from sklearn.model_selection import learning_curve
from sklearn.model_selection import StratifiedKFold
from importlib import reload
from log import Logger
from models import deep_network_estimator as dne
from models import Ensemble_hyperopt as eh
from hyperopt import fmin,tpe,hp, STATUS_OK,Trials
from hyperopt_models import parallel_ensemble as pe
from functools import reduce
import itertools
import prediction_commons
from sklearn.externals import joblib 
import get_winid

/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/ec2-user/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def plot_curves_cv(probs,label,methods,types='roc_curve'):
    plt.figure(figsize=(7,5))
    plt.title(types)
    plt.axis([0,1,0,1])
    lw = 2
    colors = ['r','b','g','k','c','m','y']
    for method,color in zip(methods,colors[:len(methods)]):
        if types == 'precision_recall_curve':       
            precision,recall,threshold = precision_recall_curve(label,probs[method])
            plt.plot(recall,precision,color,linewidth=2,label=method)
            plt.xlabel('Recall')
            plt.ylabel('Precision')
            plt.plot([0, 1], [1, 0], color='navy', lw=lw, linestyle='--')
        if types == 'roc_curve':
            fpr,tpr, threshold = roc_curve(label,probs[method])
            plt.plot(fpr,tpr,color,linewidth=2,label=method)
            plt.xlabel('False Positive Rate')
            plt.ylabel('Recall')
            plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.legend(loc='best')


#----------------------------------------------------------------------------
def get_estimators(methods,params,train_x,train_label):
    ensemble = eh.Ensemble(methods,params)
    ensemble.fit(train_x,train_label,sample_weight=sample_weight_train,max_iter=100)
    return ensemble
#-----------------------------------------------------------------------------
def plot_curves(estimators,test_x,label,types='roc_curve'):
    plt.figure(figsize=(7,5))
    plt.title(types)
    plt.axis([0,1,0,1])
    lw = 2
    colors = ['r','b','g','k','c','m','y']
    for color,estimator in zip(colors[:len(estimators)],estimators):
        name = type(estimator).__name__
        probs = np.array(estimator.predict_proba(test_x))[:,1]
        if types == 'precision_recall_curve':       
            precision,recall,threshold = precision_recall_curve(label,probs)
            plt.plot(recall,precision,color,linewidth=2,label=name)
            plt.xlabel('Recall')
            plt.ylabel('Precision')
            plt.plot([0, 1], [1, 0], color='navy', lw=lw, linestyle='--')
        if types == 'roc_curve':
            fpr,tpr, threshold = roc_curve(label,probs)
            plt.plot(fpr,tpr,color,linewidth=2,label=name)
            plt.xlabel('False Positive Rate')
            plt.ylabel('Recall')
            plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.legend(loc='best')
#-----------------------------------------------------------------------------
def learn_curve(model,train_x,train_label,cv=3,scoring='neg_log_loss'):
    model_c = clone(model)
    N,train_score,test_score = learning_curve(model_c, 
                                            train_x,train_label,cv=cv,train_sizes=np.linspace(0.5,1,5),
                                            scoring=scoring)
    
    plt.figure(figsize=(7,4))
    plt.title('{}'.format(type(model).__name__))
    plt.plot(N,np.mean(train_score,1),color='blue', label='training score')
    plt.plot(N,np.mean(test_score,1),color='red',label='validation score')
    plt.xlabel('training sample')
    plt.ylabel(scoring)
    plt.legend(loc=0)
    plt.show()
#-----------------------------------------------------------------------------    
def error_analysis(estimator,test_x,label,types=['confusion_matrix']):
    print('Error analysis of ',type(estimator).__name__)
    predict = estimator.predict(test_x)
    probs = np.array(estimator.predict_proba(test_x))[:,1]
    class_num = len(label.unique())
    if 'confusion_matrix' in types:
        conf_mat = confusion_matrix(label,predict)
        row_sums = conf_mat.sum(axis=1,keepdims=True)
        norm_conf_mat = conf_mat/row_sums
        np.fill_diagonal(norm_conf_mat,0)
        plt.matshow(norm_conf_mat,cmap=plt.cm.gray)        
    if 'precision_recall_curve' in types and class_num<=2:
        precision,recall,threshold = precision_recall_curve(label,probs)
        plot_curve(recall,precision,type(estimator).__name__,'precision_recall_curve')
    if 'roc_curve' in types and class_num<=2:
        fpr,tpr, threshold = roc_curve(label,probs)
        plot_curve(fpr,tpr,type(estimator).__name__,'roc_curve')    
    plt.show()
#---------------------------------------------------------------------------    
def plot_curve(score1,score2,label,types):
    plt.figure(figsize=(7,5))
    plt.title(types)
    plt.plot(score1,score2,linewidth=2,label=label)
    plt.plot([0,1],[1,1],'k--')
    plt.axis([0,1,0,1])
    lw = 2
    if types == 'precision_recall_curve':        
        plt.xlabel('Recall')
        plt.ylabel('Precision')
        plt.plot([0, 1], [1, 0], color='navy', lw=lw, linestyle='--')
    if types == 'roc_curve':
        plt.xlabel('False Positive Rate')
        plt.ylabel('Recall')
        plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
#--------------------------------------------------------------------------
def scores(y,predicts,pred_probs,average='macro'):
    class_num = len(y.unique())
    score_map = {}

    if class_num <=2:
        recall = recall_score(y,predicts)      
        precision = precision_score(y,predicts)      
        accuracy = accuracy_score(y,predicts)
        f1 = f1_score(y,predicts)
        auc = roc_auc_score(y,pred_probs)
        score_map['auc'] = auc
    else:
        recall = recall_score(y,predicts,average=average)      
        precision = precision_score(y,predicts,average=average)      
        accuracy = accuracy_score(y,predicts)
        f1 = f1_score(y,predicts,average=average)
        
    score_map['recall'] = recall
    score_map['precision'] = precision
    score_map['accuracy'] = accuracy
    score_map['f1'] = f1
    return score_map

#---------------------------------------------------------------------------
def cross_val_ensemble(x,y,sample_weight,methods,params,fold=10, hyperopt=True,up_sampling=False):
    skfolds = StratifiedKFold(n_splits=10,random_state=43)
    results = []
    model_combine_scores_cv = []
    model_scores_cv = []
    best_params_cv = []
    cv_columns = ['label','ensemble']+methods
    pred_probs_cv = pd.DataFrame(columns=cv_columns)
    predicts_cv = pd.DataFrame(columns=cv_columns)
    if not hyperopt:
        search_methods = get_train_models(models=methods)
    class_num = len(y.unique())
    for train_index,test_index in skfolds.split(x,y):
        train_fold = x.ix[train_index,:]
        train_label = y[train_index]
        sample_weight_train = sample_weight[train_index]
        test_fold = x.ix[test_index,:]
        test_label = y[test_index]
        sample_weight_test = sample_weight[test_index]
        if up_sampling:
            train_fold,train_label,sample_weight_train = upsampling(train_fold,train_label,sample_weight_train,fold=9)
            test_fold,test_label,sample_weight_test = downsampling(test_fold,test_label,sample_weight_test)
        if not hyperopt:            
            for param_l in params.values():
                param = param_l[0]
                if 'sample_weight' in param:
                    param['sample_weight'] = [sample_weight_train,]
            ensemble = es.Ensemble(methods=search_methods,params=params)
            ensemble.fit(train_fold,train_label)            
        else:
            ensemble = eh.Ensemble(methods,params)
            ensemble.fit(train_fold,train_label,sample_weight=sample_weight_train,max_iter=100)
        score = ensemble.score(test_fold,test_label)
        results.extend([score])
        model_prob = ensemble.model_probs()
        model_preds = ensemble.model_predicts() 
        ensemble_prob = ensemble.predict_proba(test_fold)
        ensemble_pred = ensemble.predict(test_fold)
        
        temp_df = pd.DataFrame(columns=pred_probs_cv.columns)
        for method,prob in model_prob.items():
            temp_df[method] = prob[:,1]
            temp_df[method] = temp_df[method].astype('f')
        temp_df['ensemble'] = ensemble_prob[:,1]
        temp_df['ensemble'] = temp_df['ensemble'].astype('f')
        temp_df['label'] = test_label
        temp_df['label'] = temp_df['label'].astype('i8')
        pred_probs_cv = pred_probs_cv.append(temp_df,ignore_index=True)
        
        
        temp_df = pd.DataFrame(columns=predicts_cv.columns)
        for method,pred in model_preds.items():
            temp_df[method] = pred
            temp_df[method] = temp_df[method].astype('i8')
        temp_df['ensemble'] = ensemble_pred
        temp_df['ensemble'] = temp_df['ensemble'].astype('i8')
        temp_df['label'] = test_label
        temp_df['label'] = temp_df['label'].astype('i8')
        predicts_cv = predicts_cv.append(temp_df,ignore_index=True)
        
        comb_results = methods_combination_results(methods,model_prob,test_label)
        model_combine_scores_cv.extend([comb_results])
        model_score = ensemble.get_model_scores()
        model_scores_cv.extend([model_score.copy()])
        best_params = ensemble.best_params()
        best_params_cv.extend([best_params.copy()])
        all_estimators = list(ensemble.best_estimators_.values())
        all_estimators.extend([ensemble])
        plot_curves(all_estimators,test_fold,test_label,types='roc_curve')
        plot_curves(all_estimators,test_fold,test_label,types='precision_recall_curve')
        del ensemble
    if class_num == 2:
        result_df = pd.DataFrame(results,columns=['logloss','f1','recall','precision','auc_score'])
    else:
        result_df = pd.DataFrame(results,columns=['logloss','f1','recall','precision'])
    return result_df,model_combine_scores_cv,model_scores_cv,best_params_cv,pred_probs_cv,predicts_cv


#--------------------------------------------------------------------------------------------------------------------
def soft_voting(model_probs):
    pred_probs = reduce(lambda x,y: np.add(x,y), model_probs.values())/len(model_probs)
    print(pred_probs)
    predicts = np.argmax(pred_probs,axis=1)
    return pred_probs,predicts    


#-----------------------------------------------------------------------------------
def get_train_models(models=['LogisticRegression','RandomForestClassifier','SVC','MLPClassifier','xgbooster','tensor_DNN']):
    methods = []
    l = LogisticRegression()
    rf = RandomForestClassifier()
    svc = SVC()
    xg = xgbooster.xgbooster()
    mlp = MLPClassifier()
    dnn = dne.tensor_DNN()
    lsvc = LinearSVC()
    if 'LogisticRegression' in models:
        methods.extend([l])
    if 'RandomForestClassifier' in models:
        methods.extend([rf])
    if 'SVC' in models:
        methods.extend([svc])
    if 'MLPClassifier' in models:
        methods.extend([mlp])
    if 'xgbooster' in models:
        methods.extend([xg])
    if 'tensor_DNN' in models:
        methods.extend([dnn])
    if 'LinearSVC' in models:
        methods.extend([lsvc])
    return methods
 #------------------------------------------------------------------------------

def get_hyperopt_params(methods=['LogisticRegression','RandomForestClassifier','LinearSVC','SVC','xgbooster','tensor_DNN','MLPClassifier'],wtf_lo=1,wtf_hi=1):
    weight_factor = hp.uniform('weight_factor',wtf_lo,wtf_hi)
    weight_factor = hp.uniform('weight_factor',wtf_lo,wtf_hi)
    params={}
    l_param = {'C': hp.uniform('C',0.05,20),'weight_factor':weight_factor}
    rf_param = {'n_estimators':100+hp.randint('n_estimators',900),'max_depth':5+hp.randint('max_depth',20), 'min_samples_split': 5+hp.randint('min_samples_split',15),'min_samples_leaf': 2+hp.randint('min_samples_leaf',4),'weight_factor':weight_factor}
    svc_param = {'C': hp.uniform('C',0.005,1),'gamma': hp.uniform('gamma',0.001,1),'probability':hp.choice('probability',[True]),'weight_factor':weight_factor}
    xgb_param = {'learning_rate':hp.choice('learning_rate',[0.1]),'max_depth': 5+hp.randint('max_depth',15),'n_estimators':500+hp.randint('n_estimators',2000),'reg_lambda': hp.uniform('reg_lambda',20,100),'gamma': hp.uniform('gamma',0.01,10),'weight_factor':weight_factor}
    dnn_param = {'batch_normalization': hp.choice('batch_normalization',[True]),
                 'l2_reg': hp.uniform('l2_reg',0.001,5),                            
                 'drop_out':hp.uniform('drop_out',0.1,0.8),
                 'weight_factor':weight_factor,
                 'steps':200+hp.randint('steps',1000),
                 'batch_size':hp.choice('batch_size',[30]),
                 'scoring':hp.choice('scoring',['precision']),
                 }
    mlp_param = {'alpha':hp.uniform('alpha',0.001,5),'max_iter':2000+hp.randint('max_iter',1000)}
    lsvc_param = {'C': hp.uniform('C',0.1,10),'weight_factor':weight_factor} 
    if 'LogisticRegression' in methods:
        params['LogisticRegression'] = l_param
    if 'RandomForestClassifier' in methods:
        params['RandomForestClassifier'] = rf_param
    if 'SVC' in methods:
        params['SVC'] = svc_param
    if 'xgbooster' in methods:
        params['xgbooster'] = xgb_param
    if 'tensor_DNN' in methods:
        params['tensor_DNN'] = dnn_param
    if 'MLPClassifier' in methods:
        params['MLPClassifier'] = mlp_param
    if 'LinearSVC' in methods:
        params['LinearSVC'] = lsvc_param
    return params    

#----------------------------------------------------------------------
def methods_combination_results(methods,model_probs,test_label):
    n = len(methods)
    results = {}
    for i in range(1,n+1):
        iterator = itertools.combinations(methods,i)
        for combination in iterator:
            key = reduce(lambda x,y: x+'-'+y,combination)
            print(key)
            test_model_probs = {method:prob for method,prob in model_probs.items() if method in combination}
            pred_probs,pred = soft_voting(test_model_probs)
            test_score = scores(test_label,pred,pred_probs[:,1])
            results[key] = test_score.copy()
    return results
#---------------------------------------------------------------------
def upsampling(train_x,train_label,sample_weights_train,fold=9):
    trainx = train_x.copy()
    trainx['label'] = train_label
    trainx['weight'] = sample_weights_train
    up_samples = commons.upSampling(trainx[trainx['label']==1],10)
    trainx = trainx.append(up_samples,ignore_index=True).sample(frac=1).reset_index(drop=True)
    train_label = trainx['label']
    sample_weights_train = trainx['weight']
    trainx = trainx.drop(['label','weight'],axis=1)
    return trainx,train_label,sample_weights_train

#------------------------------------------------------------------------
def downsampling(x,y,sample_weights):
    x = x.copy()
    x['label'] = y
    x['weight'] = sample_weights
    pos = x[x['label']==1]
    negs = x[x['label']==0].sample(pos.shape[0])
    alls = pos.append(negs,ignore_index=True)
    label = alls['label']
    weights = alls['weight']
    return alls.drop(['label','weight'],axis=1),label,weights

#---------------------------------------------------------------------------
def nearest_tss(tss,sites_df):
    merged = pd.merge(sites_df,tss,how='outer',on=['chr','coordinate'])
    merged.sort_values(['chr','coordinate'],inplace=True)
    merged.rename(columns={'strand':'before_tss'},inplace=True)
    merged.ix[merged['before_tss'].isnull()==False, 'before_tss'] = merged.ix[merged['before_tss'].isnull()==False,'coordinate']
    merged['after_tss'] = merged['before_tss']
    merged['before_tss'].fillna(method='ffill', inplace=True)
    merged['after_tss'].fillna(method='bfill',inplace=True)
    merged['dist_to_before_tss'] = np.abs(merged['coordinate']-merged['before_tss'])
    merged['dist_to_after_tss'] = np.abs(merged['coordinate']-merged['after_tss'])
    merged['tss'] = None
    before_ix = (merged['dist_to_before_tss'] < merged['dist_to_after_tss']) | (merged['dist_to_after_tss'].isnull())
    merged.ix[before_ix,'tss'] = merged.ix[before_ix,'before_tss']
    after_ix = (merged['dist_to_before_tss'] >= merged['dist_to_after_tss']) | (merged['dist_to_before_tss'].isnull())
    merged.ix[after_ix,'tss'] = merged.ix[after_ix,'after_tss']
    merged['dist_to_nearest_tss'] = np.abs(merged['coordinate']-merged['tss']) 
    return merged.dropna()
    

In [3]:
dataset = 'AD_CpG'
type_name = commons.type_name  ## amyloid, cerad, tangles
with_cell_type = commons.with_cell_type ## with or without
dataset = dataset+'/'+type_name+with_cell_type
start = str(prediction_commons.tss_start)
end = str(prediction_commons.tss_end)
all_features = home+'data/WGBS/all_features_'+str(start)+'_'+str(end)
model_path = home+'data/'+dataset+'/prediction_model.pkl'
pred_probs = home+'data/'+dataset+'/pred_probs'

In [4]:
start

'0'

In [6]:
##features selecetd by traditional methods

up_sampling = True;
if up_sampling:
    wtf_lo = 0.05 if dataset=="RICHS" else 0.2
    wtf_hi = 0.1 if dataset=="RICHS" else 0.3
else:
    wtf_lo = 1.0/3 if dataset=="RICHS" else 1 
    wtf_hi = 0.5 if dataset=="RICHS" else 1.5
    
log_dir = home+'logs/'
logger = Logger.Logger(log_dir,False).get_logger()
with pd.HDFStore(home+'data/'+dataset+'/selected_features','r') as h5s:
    train_x =h5s['train_x'] 
    train_label = h5s['train_label'] 
    test_x = h5s['test_x'] 
    test_label = h5s['test_label']
    sample_weights_train = h5s['sample_weights_train'] 
    sample_weights_test = h5s['sample_weights_test']
total_x = pd.concat([train_x,test_x],ignore_index=True)
total_label = pd.concat([train_label,test_label],ignore_index=True)
total_sample_weights = pd.concat([sample_weights_train,sample_weights_test],ignore_index=True)

In [7]:
######model training
methods = ['LogisticRegression','xgbooster']
params = get_hyperopt_params(methods,wtf_lo=wtf_lo,wtf_hi=wtf_hi)
ensemble_hyopt = eh.Ensemble(methods,params)
ensemble_hyopt.fit(total_x,total_label,sample_weight=total_sample_weights,max_iter=100)
joblib.dump(ensemble_hyopt,model_path)

0.2620749022115186
0.2056341773792153
0.23404755175461578
0.2026276140565031
0.20876476232430796
0.27443152547097316
0.21373285713525667
0.29825596866709064
0.22030043601383092
0.22270981401393183
0.2004684048258926
0.27870502623199744
0.23166631543122052
0.2850381441711615
0.21740653714152494
0.2147397605860217
0.28087813395178035
0.2160327322138077
0.24607160948298243
0.23390606206484751
0.24825025172305726
0.22581589315382383
0.24170962338492732
0.2611473499053313
0.25796765231415614
0.21193476934090458
0.23914170674818183
0.2248823540010259
0.2264435992927098
0.228790626111486


/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


0.25662042445931843
0.2669391067685023
0.25280933618807144
0.24172575789263098
0.20489629597447748
0.20785963289425452
0.23786013089081104


/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


0.22605793170535
0.22157488918287008
0.21007003465993646
0.26911871464157394
0.20202914652799306
0.22998139339067117
0.29034454087803846
0.2367251066437235
0.24408175224701578
0.2193184530724099
0.25034713871540304
0.20038522842853493
0.2240107188310674
0.21373108303102784
0.23519681519385097
0.205980722734947
0.21828115875672707
0.23087824917144506
0.2995686766778947
0.25265948292067564
0.26677297224220653
0.2272933716369282
0.24465808636012343
0.2327413270988937
0.274512598137124
0.22328782672173456
0.2414079431960537
0.21053651885035388
0.24057884921039796
0.2560388599734579
0.248575083319477
0.21514673749543167
0.2609162380341462


...

/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

0.2515787026778691
{'base_score': 0.5, 'booster': 'gbtree', 'colsample_bylevel': 1, 'colsample_bytree': 1, 'gamma': 2.533293090323736, 'learning_rate': 0.1, 'max_delta_step': 0, 'max_depth': 12, 'min_child_weight': 1, 'missing': None, 'n_estimators': 1041, 'n_jobs': -1, 'nthread': None, 'objective': 'binary:logistic', 'random_state': 0, 'reg_alpha': 0, 'reg_lambda': 42.26058781578625, 'scale_pos_weight': 1, 'seed': None, 'silent': True, 'subsample': 1}
{'base_score': 0.5, 'booster': 'gbtree', 'colsample_bylevel': 1, 'colsample_bytree': 1, 'gamma': 2.533293090323736, 'learning_rate': 0.1, 'max_delta_step': 0, 'max_depth': 12, 'min_child_weight': 1, 'missing': None, 'n_estimators': 1041, 'n_jobs': -1, 'nthread': None, 'objective': 'binary:logistic', 'random_state': 0, 'reg_alpha': 0, 'reg_lambda': 42.26058781578625, 'scale_pos_weight': 1, 'seed': None, 'silent': True, 'subsample': 1}
{'base_score': 0.5, 'booster': 'gbtree', 'colsample_bylevel': 1, 'colsample_bytree': 1, 'gamma': 2.533293

/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: Deprec

{'gamma': 0.5043999420760845, 'learning_rate': 0, 'max_depth': 1, 'n_estimators': 33, 'reg_lambda': 53.88526379571991, 'weight_factor': 0.27198425316317093}


['/home/ec2-user/git/EnsembleCpG/data/AD_CpG/cogdecwith/prediction_model.pkl']

In [8]:
selected_features = total_x.columns
with pd.HDFStore(all_features,'r') as h5s:
    wgbs_all_data = h5s['all_features']
wgbs_data = wgbs_all_data[selected_features]

In [9]:
ensemble_hyopt = joblib.load(model_path)
for method,best_estimator in ensemble_hyopt.best_estimators_.items():
    best_estimator.fit(total_x,total_label,total_sample_weights)
pred = ensemble_hyopt.predict(wgbs_data)
pred_prob = ensemble_hyopt.predict_proba(wgbs_data)

{'base_score': 0.5, 'booster': 'gbtree', 'colsample_bylevel': 1, 'colsample_bytree': 1, 'gamma': 0.5043999420760845, 'learning_rate': 0.1, 'max_delta_step': 0, 'max_depth': 6, 'min_child_weight': 1, 'missing': None, 'n_estimators': 533, 'n_jobs': -1, 'nthread': None, 'objective': 'binary:logistic', 'random_state': 0, 'reg_alpha': 0, 'reg_lambda': 53.88526379571991, 'scale_pos_weight': 1, 'seed': None, 'silent': True, 'subsample': 1, 'weight_factor': 0.27198425316317093}


/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [10]:
pred_prob = pd.DataFrame(pred_prob,columns=['negative','positive'])
target_sites = pred_prob.sort_values(['positive'],ascending=False).query('positive >= 0.5')
target_sites_coordinate = wgbs_all_data.ix[target_sites.index,['chr','coordinate']]
target_sites = target_sites.join(target_sites_coordinate)
with pd.HDFStore(pred_probs,'a') as h5s:
    h5s[start+'_'+end] = target_sites

/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  app.launch_new_instance()
/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/site-packages/tables/path.py:112: NaturalNameWarning: object name is not a valid Python identifier: '0_1000000'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  NaturalNameWarning)


In [11]:
with pd.HDFStore(pred_probs,'r') as h5s:
     target_sites1 = h5s[start+'_'+end] 

In [12]:
target_sites1

,negative,positive,chr,coordinate
785457,0.015673,0.984327,1,110041388.0
220419,0.015733,0.984267,1,15748452.0
785455,0.016764,0.983236,1,110041309.0
785456,0.018168,0.981832,1,110041326.0
220420,0.018675,0.981325,1,15748494.0
220418,0.018675,0.981325,1,15748417.0
429894,0.019031,0.980969,1,33735090.0
455377,0.019054,0.980946,1,36709562.0
455364,0.019054,0.980946,1,36709404.0
455378,0.019054,0.980946,1,36709566.0


In [13]:
with pd.HDFStore(pred_probs,'r') as h5s:
    print(h5s.keys())

['/0_1000000']


In [14]:
pred_probs

'/home/ec2-user/git/EnsembleCpG/data/AD_CpG/cogdecwith/pred_probs'

In [5]:
first = True
with pd.HDFStore(pred_probs,'r') as h5s:
    for key in h5s.keys():
        if first:
            top_500 = h5s[key][:500]
            first = False
        else:
            top_500 = pd.concat([top_500,h5s[key][:500]],ignore_index=True)

In [12]:
top_500 = top_500.sort_values(['positive'],ascending=False).reset_index(drop=True)
top_500 = top_500[:500]
top_500['coordinate'] = top_500['coordinate'].astype('i8')

In [13]:
top_500

,negative,positive,chr,coordinate
0,0.017688,0.982312,13,108867204
1,0.018883,0.981117,13,108867297
2,0.018883,0.981117,13,108867272
3,0.018883,0.981117,13,108867228
4,0.019672,0.980328,1,101701495
5,0.020166,0.979834,12,120806943
6,0.021424,0.978576,19,798382
7,0.021703,0.978297,12,120806949
8,0.021897,0.978103,16,6069326
9,0.021897,0.978103,16,6069356


In [14]:
#nearest tss distance    
chrs = top_500['chr'].unique()
cols=['chr', 'coordinate','strand']
tss =  pd.read_csv(home+'tss.txt',sep='\s+',header=None,names=cols,skiprows=1)
tss = get_winid.convert_chr_to_num(tss,chrs)
tss.sort_values(['chr','coordinate'],inplace=True)
top_500 = nearest_tss(tss,top_500).sort_values(['positive'],ascending=False)

['13', '1', '12', '19', '16', '14', '11', '20', '18', '10', '17', '15', '21']
1     chr1
2     chr1
3     chr1
4     chr1
5     chr1
6     chr1
7     chr1
8     chr1
9     chr1
10    chr1
Name: chr, dtype: object


/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/site-packages/ipykernel/__main__.py:316: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [15]:
pred_positive_500 = home+'data/'+dataset+'/pred_positive_500.csv'
top_500.to_csv(pred_positive_500,index=False)
pred_positive_500_bed = home+'data/'+dataset+'/pred_positive_500.bed'
top_500_bed = top_500.copy()
top_500_bed['end'] = top_500_bed['coordinate']+1
top_500_bed['chr'] = top_500_bed['chr'].apply(lambda x: 'chr'+str(x))
top_500_bed[['chr','coordinate','end','positive']].to_csv(pred_positive_500_bed,index=False,header=None,sep='\t')

In [15]:
pred_probs = [home+'data/'+dataset+'/pred_probs' for dataset in ['AD_CpG/amyloidwith','AD_CpG/ceradwith','AD_CpG/tangleswith']]
traits = ['amyloidwith','ceradwith','tangleswith']
three_dataset_common_predictions = None
for pred_prob,trait in zip(pred_probs,traits):
    first = True
    with pd.HDFStore(pred_prob,'r') as h5s:
        for key in h5s.keys():
            if first:
                predicted_positives = h5s[key]
                first = False
            else:
                predicted_positives = pd.concat([predicted_positives,h5s[key]],ignore_index=True)
    if three_dataset_common_predictions is None:
        three_dataset_common_predictions = predicted_positives
    else:
        three_dataset_common_predictions = three_dataset_common_predictions.merge(predicted_positives,on=['chr','coordinate'],how='inner')
    three_dataset_common_predictions.rename({'positive':'positive_'+trait,'negative':'negative_'+trait},axis=1,inplace=True)

In [16]:
three_dataset_common_predictions

,negative_amyloidwith,positive_amyloidwith,chr,coordinate,negative_ceradwith,positive_ceradwith,negative_tangleswith,positive_tangleswith
0,0.147724,0.852276,1,144612725.0,0.299895,0.700105,0.218465,0.781535
1,0.147725,0.852275,1,144612710.0,0.299925,0.700075,0.218465,0.781535
2,0.165912,0.834088,1,156863586.0,0.264140,0.735860,0.332752,0.667248
3,0.168157,0.831843,1,113466014.0,0.178786,0.821214,0.196226,0.803774
4,0.168574,0.831426,1,113466051.0,0.171304,0.828696,0.205575,0.794425
5,0.168615,0.831385,1,113466061.0,0.171304,0.828696,0.205575,0.794425
6,0.169476,0.830524,1,156863542.0,0.278163,0.721837,0.430138,0.569862
7,0.169680,0.830320,1,156863492.0,0.334671,0.665329,0.405665,0.594335
8,0.170683,0.829317,1,113466022.0,0.178786,0.821214,0.198841,0.801159
9,0.170898,0.829102,1,156863503.0,0.193439,0.806561,0.365032,0.634968


In [17]:
three_dataset_common_predictions['mean_positive'] = three_dataset_common_predictions[['positive_'+trait for trait in traits]].mean(axis=1)
tenfold_test_results = {}
weight_scale = 'f1' #'auc','recall','precision','f1','accuracy'
scale_sum = 0
weight_score = None
for trait in traits:
    trait_result = joblib.load(home+'data/'+dataset+'/10fold_test_results.pkl')
    tenfold_test_results[trait] = trait_result
    three_dataset_common_predictions['weight_'+weight_scale+'_'+trait] = trait_result[weight_scale]
    if weight_score is None:
        weight_score = three_dataset_common_predictions['positive_'+trait]*trait_result[weight_scale]
    else:
        weight_score += three_dataset_common_predictions['positive_'+trait]*trait_result[weight_scale]
three_dataset_common_predictions['weighted_positive'] = weight_score

KeyError: 'f1'

In [17]:
three_dataset_common_predictions.sort_values(['weighted_positive'],inplace=True,ascending=False)

In [18]:
three_dataset_common_predictions_top500 = three_dataset_common_predictions[:500].drop(['negative','negative_x','negative_y'],axis=1)

In [19]:
three_dataset_common_predictions_top500

,positive_amyloidwith,chr,coordinate,positive_ceradwith,positive_tangleswith,mean_positive
7300,0.827166,15,25438455.0,0.954520,0.879063,0.886916
7298,0.836591,15,25438539.0,0.948615,0.862548,0.882585
7299,0.828682,15,25438472.0,0.951225,0.863962,0.881290
7327,0.804757,15,25438423.0,0.953638,0.885154,0.881183
7337,0.797714,15,25438443.0,0.952086,0.887319,0.879039
7344,0.796293,15,25438403.0,0.952546,0.887319,0.878719
14853,0.811396,21,45978677.0,0.975623,0.847657,0.878225
3973,0.800389,11,1995456.0,0.884854,0.948316,0.877853
7301,0.825712,15,25438446.0,0.918937,0.883890,0.876180
12356,0.758293,19,35630355.0,0.955051,0.909660,0.874335


In [20]:
#nearest tss distance    
chrs = three_dataset_common_predictions_top500['chr'].unique()
cols=['chr', 'coordinate','strand']
tss =  pd.read_csv(home+'tss.txt',sep='\s+',header=None,names=cols,skiprows=1)
tss = get_winid.convert_chr_to_num(tss,chrs)
tss.sort_values(['chr','coordinate'],inplace=True)
three_dataset_common_predictions_top500 = nearest_tss(tss,three_dataset_common_predictions_top500).sort_values(['mean_positive'],ascending=False)

['15', '21', '11', '19', '16', '17', '12', '14', '13', '1', '10', '20']
1     chr1
2     chr1
3     chr1
4     chr1
5     chr1
6     chr1
7     chr1
8     chr1
9     chr1
10    chr1
Name: chr, dtype: object


/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/site-packages/ipykernel/__main__.py:316: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [21]:
three_dataset_common_predictions_top500

,positive_amyloidwith,chr,coordinate,positive_ceradwith,positive_tangleswith,mean_positive,before_tss,after_tss,dist_to_before_tss,dist_to_after_tss,tss,dist_to_nearest_tss
0,0.827166,15,25438455.0,0.954520,0.879063,0.886916,25436566.0,25438468.0,1889.0,13.0,25438468,13.0
1,0.836591,15,25438539.0,0.948615,0.862548,0.882585,25438468.0,25440068.0,71.0,1529.0,25438468,71.0
2,0.828682,15,25438472.0,0.951225,0.863962,0.881290,25438468.0,25440068.0,4.0,1596.0,25438468,4.0
3,0.804757,15,25438423.0,0.953638,0.885154,0.881183,25436566.0,25438468.0,1857.0,45.0,25438468,45.0
4,0.797714,15,25438443.0,0.952086,0.887319,0.879039,25436566.0,25438468.0,1877.0,25.0,25438468,25.0
5,0.796293,15,25438403.0,0.952546,0.887319,0.878719,25436566.0,25438468.0,1837.0,65.0,25438468,65.0
6,0.811396,21,45978677.0,0.975623,0.847657,0.878225,45978643.0,45993606.0,34.0,14929.0,45978643,34.0
7,0.800389,11,1995456.0,0.884854,0.948316,0.877853,1968502.0,2011150.0,26954.0,15694.0,2011150,15694.0
8,0.825712,15,25438446.0,0.918937,0.883890,0.876180,25436566.0,25438468.0,1880.0,22.0,25438468,22.0
9,0.758293,19,35630355.0,0.955051,0.909660,0.874335,35629732.0,35630392.0,623.0,37.0,35630392,37.0


In [22]:
common_top_500 = home+'data/AD_CpG/pred_positive_500_three_commmons.csv'
three_dataset_common_predictions_top500.to_csv(common_top_500,index=False)

In [6]:
######predict 450k array sites
pred_probs_450k = home+'data/'+dataset+'/pred_probs_450k'
selected_features = total_x.columns
all_450k_features = home+'data/AD_CpG/all_450k_features'
with pd.HDFStore(all_450k_features,'r') as h5s:
    all_450k_data = h5s['all_450k_features']
all_450k_data_features = all_450k_data[selected_features]


In [7]:
ensemble_hyopt = joblib.load(model_path)
for method,best_estimator in ensemble_hyopt.best_estimators_.items():
    best_estimator.fit(total_x,total_label,total_sample_weights)
pred_450k = ensemble_hyopt.predict(all_450k_data_features)
pred_prob_450k = ensemble_hyopt.predict_proba(all_450k_data_features)

{'base_score': 0.5, 'booster': 'gbtree', 'colsample_bylevel': 1, 'colsample_bytree': 1, 'gamma': 0.017804821807843686, 'learning_rate': 0.1, 'max_delta_step': 0, 'max_depth': 13, 'min_child_weight': 1, 'missing': None, 'n_estimators': 2040, 'n_jobs': -1, 'nthread': None, 'objective': 'binary:logistic', 'random_state': 0, 'reg_alpha': 0, 'reg_lambda': 21.982717075904347, 'scale_pos_weight': 1, 'seed': None, 'silent': True, 'subsample': 1, 'weight_factor': 0.28027509010842716}


/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [8]:
pred_prob_450k = pd.DataFrame(pred_prob_450k,columns=['negative','positive'])
pred_prob_450k['chr'] = all_450k_data['chr']
pred_prob_450k['coordinate'] = all_450k_data['coordinate']

In [9]:
with pd.HDFStore(pred_probs_450k,'w') as h5s:
    h5s['pred_probs_450k'] = pred_prob_450k

In [15]:
with pd.HDFStore(pred_probs_450k,'r') as h5s:
     pred_prob_450k = h5s['pred_probs_450k'] 

In [18]:
pred_probs = [home+'data/'+dataset+'/pred_probs_450k' for dataset in ['AD_CpG/amyloidwith','AD_CpG/ceradwith','AD_CpG/tangleswith']]
traits = ['amyloidwith','ceradwith','tangleswith']
first = True
for pred_prob,trait in zip(pred_probs,traits):
    with pd.HDFStore(pred_prob,'r') as h5s:
        if first:
            all_probs_450k = h5s['pred_probs_450k']
            first = False
        else:
            all_probs_450k = pd.concat([all_probs_450k,h5s['pred_probs_450k']],axis=1)
    all_probs_450k.rename({'positive':'positive_'+trait,'negative':'negative_'+trait},axis=1,inplace=True)

In [21]:
all_probs_450k = all_probs_450k.ix[:,~all_probs_450k.columns.duplicated()]

In [24]:
all_probs_450k = pd.merge(all_probs_450k,all_450k_data[['id','chr','coordinate','pvalue','beta']],on=['chr','coordinate'],how='left')

In [25]:
all_probs_450k['mean_positive'] = all_probs_450k[['positive_'+trait for trait in traits]].mean(axis=1)

In [27]:
all_probs_450k.to_csv(home+'data/AD_CpG/450kwithpredictedprob.csv',index=False)

In [46]:
common_top_500 = home+'data/AD_CpG/pred_positive_500_three_commmons.csv'
three_dataset_common_predictions_top500 = pd.read_csv(common_top_500)

In [47]:
three_dataset_common_predictions_top500

,positive_amyloidwith,chr,coordinate,positive_ceradwith,positive_tangleswith,mean_positive,before_tss,after_tss,dist_to_before_tss,dist_to_after_tss,tss,dist_to_nearest_tss
0,0.827166,15,25438455.0,0.954520,0.879063,0.886916,25436566.0,25438468.0,1889.0,13.0,25438468,13.0
1,0.836591,15,25438539.0,0.948615,0.862548,0.882585,25438468.0,25440068.0,71.0,1529.0,25438468,71.0
2,0.828682,15,25438472.0,0.951225,0.863962,0.881290,25438468.0,25440068.0,4.0,1596.0,25438468,4.0
3,0.804757,15,25438423.0,0.953638,0.885154,0.881183,25436566.0,25438468.0,1857.0,45.0,25438468,45.0
4,0.797714,15,25438443.0,0.952086,0.887319,0.879039,25436566.0,25438468.0,1877.0,25.0,25438468,25.0
5,0.796293,15,25438403.0,0.952546,0.887319,0.878719,25436566.0,25438468.0,1837.0,65.0,25438468,65.0
6,0.811396,21,45978677.0,0.975623,0.847657,0.878225,45978643.0,45993606.0,34.0,14929.0,45978643,34.0
7,0.800389,11,1995456.0,0.884854,0.948316,0.877853,1968502.0,2011150.0,26954.0,15694.0,2011150,15694.0
8,0.825712,15,25438446.0,0.918937,0.883890,0.876180,25436566.0,25438468.0,1880.0,22.0,25438468,22.0
9,0.758293,19,35630355.0,0.955051,0.909660,0.874335,35629732.0,35630392.0,623.0,37.0,35630392,37.0


In [48]:
with pd.HDFStore(home+'data/AD_CpG/all_450k_sites_winid','r') as h5s:
    all_sites = h5s['all_450k_sites_winid']
top500_nearest_cpgs = commons.find_nearest_450ksites(5000,all_sites,three_dataset_common_predictions_top500)

/home/ec2-user/git/EnsembleCpG/code/common/commons.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  nearby_sites['wgbs_chr'] = chr
/home/ec2-user/git/EnsembleCpG/code/common/commons.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  nearby_sites['wgbs_coordinate'] = site['coordinate']


In [49]:
top500_nearest_cpgs

id  chr  coordinate    pvalue      beta  \
wgbs_chr wgbs_coordinate                                                    
15.0     25438455.0       cg18094401   15  25434150.0  0.348795  0.679325   
         25438455.0       cg08042710   15  25434475.0  0.866122  0.725468   
         25438455.0       cg11148364   15  25434492.0  0.632711  0.653330   
         25438455.0       cg18266456   15  25434500.0  0.444992  0.934735   
         25438455.0       cg04060551   15  25435100.0  0.406721  0.794604   
         25438455.0       cg14002752   15  25435226.0  0.560489  0.656645   
         25438455.0       cg27204266   15  25436114.0  0.757109  0.446847   
         25438455.0       cg03188200   15  25436244.0  0.927731  0.975220   
         25438455.0       cg25908613   15  25436246.0  0.224766  0.908265   
         25438455.0       cg07144689   15  25436496.0  0.935419  0.665176   
         25438455.0       cg08564375   15  25436682.0  0.676393  0.648712   
         25438455.0       cg26958654   15  25437871.0  0.202977  0.706798   
         25438455.0       cg02361459   15  25437982.0  0.561489  0.640327   
         25438455.0       cg15479427   15  25438173.0  0.382428  0.554055   
         25438455.0       cg11273471   15  25438403.0  0.955155  0.704391   
         25438455.0       cg12919388   15  25438443.0  0.851336  0.736913   
         25438455.0       cg05230642   15  25439252.0  0.768752  0.622504   
         25438455.0       cg13403695   15  25439493.0  0.657097  0.747835   
         25438455.0       cg26397662   15  25439886.0  0.173035  0.631400   
         25438455.0       cg06263473   15  25441607.0  0.790866  0.740744   
         25438455.0       cg09779576   15  25442386.0  0.832027  0.645411   
         25438455.0       cg21220965   15  25442571.0  0.848610  0.586111   
         25438539.0       cg18094401   15  25434150.0  0.348795  0.679325   
         25438539.0       cg08042710   15  25434475.0  0.866122  0.725468   
         25438539.0       cg11148364   15  25434492.0  0.632711  0.653330   
         25438539.0       cg18266456   15  25434500.0  0.444992  0.934735   
         25438539.0       cg04060551   15  25435100.0  0.406721  0.794604   
         25438539.0       cg14002752   15  25435226.0  0.560489  0.656645   
         25438539.0       cg27204266   15  25436114.0  0.757109  0.446847   
         25438539.0       cg03188200   15  25436244.0  0.927731  0.975220   
...                              ...  ...         ...       ...       ...   
17.0     48546224.0       cg06818777   17  48546620.0  0.416363  0.119206   
         48546224.0       cg12649208   17  48548533.0  0.729755  0.524861   
         48546224.0       cg14767768   17  48549835.0  0.263764  0.860621   
19.0     35068602.0       cg13365304   19  35066283.0  0.339639  0.849261   
         35068602.0       cg25418852   19  35068221.0  0.920704  0.906850   
         35068602.0       cg23200020   19  35068555.0  0.669043  0.068213   
         35068602.0       cg07599133   19  35068628.0  0.499827  0.036392   
         35068602.0       cg04714041   19  35068710.0  0.838271  0.137330   
         35068602.0       cg02921624   19  35068788.0  0.325241  0.187997   
         35068602.0       cg08107402   19  35069269.0  0.258283  0.825042   
         35068602.0       cg07457967   19  35069613.0  0.842423  0.836450   
         49658532.0       cg00461901   19  49653797.0  0.502460  0.006581   
         49658532.0       cg02936207   19  49654234.0  0.613945  0.056872   
         49658532.0       cg15439442   19  49654467.0  0.944620  0.054367   
         49658532.0       cg08826281   19  49655176.0  0.439685  0.054611   
         49658532.0       cg17430167   19  49655218.0  0.638459  0.097578   
         49658532.0       cg24078905   19  49655323.0  0.577477  0.387780   
         49658532.0       cg19851487   19  49655517.0  0.692789  0.473604   
         49658532.0       cg11297046   19  49657000.0  0.362612  0.766776   
         49658532.0       cg10

In [44]:
top500_nearest_cpgs.drop(['start','winid','end'],axis=1).to_csv(home+'data/AD_CpG/top500_nearest_450k.csv')